In [1]:
-- setup Jupyter notebook
:opt no-lint
:opt no-pager


-- necessary extensions & imports
{-# LANGUAGE OverloadedLabels #-}
{-# LANGUAGE FlexibleContexts #-}
{-# LANGUAGE TypeOperators #-}
{-# LANGUAGE DataKinds #-}
-- :set -F -pgmF=record-dot-preprocessor|
import Data.Row.Extra
import Control.Lens hiding ((.=))



# Part 1. Declaring types of open records

Record types are parametrized by `Row`s 

In [3]:
:kind Rec

Rec :: Row * -> *
= Rec

Basic operators for building a row:

In [2]:
:kind (.==)
:kind (.=)
:kind (.+)
:kind (<+>)

(.==) :: forall k. Symbol -> k -> Row k

(.=) :: Symbol -> * -> *

(.+) :: forall k. Row k -> Row k -> Row k

(<+>) :: * -> * -> *

Example of a row:

In [50]:
type UserRow = "id" .== Int .+ "name" .== String .+ "friendIDs" .== [Int]
:kind UserRow

UserRow :: Row *

`(.+)` is commutative and aassociative: (plus demonstration of constraint chrecking)

In [51]:
{-# LANGUAGE TypeFamilies #-}
ok = () :: UserRow ~ ("name" .== String .+ ("friendIDs" .== [Int] .+ "id" .== Int)) => ()

Let's create a User :: Type

In [53]:
type User = Rec UserRow

type User2 = Rec ("id" .== Int .+ "name" .== String .+ "friendIDs" .== [Int])

type User3 =  "id" .= Int
          <+> "name" .= String
          <+> "friendIDs" .= [Int]


-- #id = Label @"id"

bob1 :: User
bob1 = #id .== 12
    .+ #name .== "TheBob"
    .+ #friendIDs .== [12, 13]

bob2 :: User3
bob2 = #id .= 12
    <+> #name .= "TheBob"
    <+> #friendIDs .= [12, 13]

bob1 == bob2

bob1

True

#friendIDs .== [12,13] .+ #id .== 12 .+ #name .== "TheBob"

{| user = "Bob", aaa = "Kek" |}

Internal structure, we should not create such types manually

# Part 2. Creating and accessing open records

We are using overloaded labels and old operators for creating records. Constraint `Forall l Unconstrained1` can be ignored here

In [6]:
:t (.+)
:t (.==)
:t (#x .=)

(.+) :: forall (l :: Row *) (r :: Row *). Forall l Unconstrained1 => Rec l -> Rec r -> Rec (l .+ r)

(.==) :: forall (l :: Symbol) a. KnownSymbol l => Label l -> a -> Rec ('R '[ l ':-> a])

(#x .=) :: forall a. a -> Rec ('R '[ "x" ':-> a])

All field labels and types are checked at compile time. Good enough error  messages:

In [11]:
{-# LANGUAGE OverloadedStrings #-}
import Data.Aeson hiding ((.=))

toJSON $ #token .= "TOKEN<AAA>" .+ #second_token Data.Row.Extra..= "SECONDTOKEN<AAA>"

Object (fromList [("second_token",String "SECONDTOKEN<AAA>"),("token",String "TOKEN<AAA>")])

Not all fields are initialized

In [8]:
bob :: User
bob = #id .== 12

: 

Typo in a field:

In [9]:
bob :: User
bob = #id .== 12
   .+ #friends .== [] 
   .+ #name .== "Bob"

: 

Wrong field type:

In [16]:
bob :: User1
bob = #id .== 12
   .+ #name .== "Bob"
   .+ #friendIDs .== [12] 

So let's create a user:

In [18]:
bob :: User1
bob = #name .== "Bob"
   .+ #id .== 12
   .+ #friendIDs .== [13, 14]

Autogenerated show and ToJSON/FromJSON instances don't care about order of fields:

In [12]:

import Data.Aeson
import Data.ByteString.Lazy as LBS
LBS.putStr . encode $ toJSON bob
bob

{"friendIDs":[13,14],"name":"Bob","id":12}

#friendIDs .== [13,14] .+ #id .== 12 .+ #name .== "Bob"

Field accessing via Lens:

In [ ]:
bob ^. #id
view #name bob

12

"Bob"

In [19]:
f :: User1 -> (String, Int)
f u = (u ^. #name <> " #" <> show (u ^. #id), u ^. #id)

f bob

("Bob #12",12)

With record dot preprocessor, we can also write

```
f :: User -> (String, Int)
f u = (u.name <> " #" <> show u.id, u.id)
```

map (.id)

but preprocessor is not stable, space-sensetive, and some advanced updates can be nicely expressed only with lens

In [20]:
bob

#friendIDs .== [13,14] .+ #id .== 12 .+ #name .== "Bob"

In [41]:
kek = bob & #friendIDs . traverse  %~ show  

In [51]:
import Data.List
bob ^. #friendIDs . filtered (13 `elem`) 

[]

# Part 3. Advanced updating
`Overloaded labels` allows us to use records as lenses for nested, polymorphic and monadic updates:

Nested monadic update examples in `demo-perudo-server/GameLogic.hs`

# Part 4. Changing a structure

We can add new fields to records and generic types with single constructor using `+=`

In [12]:
{-# LANGUAGE DataKinds #-}
{-# LANGUAGE TypeOperators #-}
{-# LANGUAGE OverloadedLabels #-}
{-# LANGUAGE DeriveGeneric #-}
import GHC.Generics (Generic)
data Point2D = Point2D {x :: Int, y :: Int} deriving (Show, Generic)


type NamedPoint3D = "z" .= Int
                <+> "prop" .= ("name" .= String) 
                <+> Point2D 

f :: Point2D -> NamedPoint3D
f x = x 
  <+> #prop .= (#name .= "Unnamed") 
  <+> #z .= 0 

z = f $ Point2D 12 13

z{prop.name = "point z"}

: 

In [8]:
:kind! NamedPoint3D

NamedPoint3D :: *
= Rec ('R '[ "prop" ':-> Rec ('R '[ "name" ':-> [Char]]), "x" ':-> Int, "y" ':-> Int, "z" ':-> Int])

Let's create a function that adds a field `name` to record:

In [27]:
giveName :: b -> (Rec r) ->   Rec ("name" .== b .+ r)  
giveName s obj = (#name .== s)  .+ obj

:t giveName

giveName :: forall b (r :: Row *). b -> Rec r -> Rec ('R '[ "name" ':-> b] .+ r)

`.+` in inferred result type is a type family that can raise a type error if such field already exists:

In [28]:
thing = #id .== 124 
     .+ #struct .== (#aaa .== "aaa" .+ #bbb .== "bbb")
     .+ #name .== "Ken" 

thing

:t giveName "The thing" thing
giveName "The thing" thing


#id .== 124 .+ #name .== "Ken" .+ #struct .== (#aaa .== "aaa" .+ #bbb .== "bbb")

giveName "The thing" thing :: forall a. Num a => Rec ('R (("id" ':-> a) : (TypeError ...)))

: 

In [43]:

aaaaa :: Rec ("a" .== Int .// "b" .== Bool )
aaaaa = #b .= True <+> #a .= 12 


aaaaa

:t aaaaa

#a .== 12 .+ #b .== True

aaaaa :: Rec ('R '[ "a" ':-> Int, "b" ':-> Bool])

We can rename a field in struct to fix this

In [44]:
giveName "The thing" $ rename #name #oldName thing  

#id .== 124 .+ #name .== "The thing" .+ #oldName .== "Ken" .+ #struct .== (#aaa .== "aaa" .+ #bbb .== "bbb")

Also we can simply drop name field from old structure: 

In [47]:
thing .- #name

#id .== 124 .+ #struct .== (#aaa .== "aaa" .+ #bbb .== "bbb")

Suppose we have structure like this:

In [18]:
struct = #user .== bob .+ #thing .==  thing

LBS.putStr . encode $ toJSON struct

{"thing":{"struct":{"bbb":"bbb","aaa":"aaa"},"name":"Ken","id":124},"user":{"friendIDs":[13,14],"name":"Bob","id":12}}

We can split a record to two parts, or restrict to subset. Via lens we can focus on subrecords to operate on them

In [55]:
import Data.Row.Records (split)

type UserInfo = Rec ("name" .== String .+ "id" .== Int)
makeUser :: UserInfo -> [Int] -> User
makeUser u friends = u .+ #friendIDs .== friends

userInfo :: User -> UserInfo
userInfo = restrict

splitUser :: User -> (UserInfo, [Int])
splitUser u = let (a, b) = Data.Row.Records.split u in (a, b ^. #friendIDs)

splitUser bob1

(#id .== 12 .+ #name .== "TheBob",[12,13])

In [36]:
import Data.Row.Records (restrict)
struct2 :: Rec ("thing" .== Rec ("name" .== String) .+ "user" .== Rec ("name" .== String))
struct2 = struct & #thing %~ restrict
                 & #user %~ restrict

struct2

#thing .== (#name .== "Ken") .+ #user .== (#name .== "Bob")

# Part 5. Functions to and from record structures

`r .! "a" ` is either value of #a in r or TypeError. Also `.!` can be used instead of `^.` in "get field" cases for better type errors 

In [13]:
f x = x .! #aaa 

:t f

f :: forall (r :: Row *). Rec r -> r .! "aaa"

`label` allows us "zip" tuple of labels with tuple (length up to 25) of values to get a record, useful with hasql

In [57]:
lb = label (#user, #id, #val, #ttt)
:t lb

lb ("MNOP", 123, [()], 123)

lb :: forall tupleElem1 tupleElem2 tupleElem3 tupleElem4.
(tupleElem1, tupleElem2, tupleElem3, tupleElem4) -> Rec ('R '[ "id" ':-> tupleElem2, "ttt" ':-> tupleElem4, "user" ':-> tupleElem1, "val" ':-> tupleElem3])

#id .== 123 .+ #ttt .== 123 .+ #user .== "MNOP" .+ #val .== [()]

In [ ]:
instance X a => C a 

